In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#import warnings
#warnings.filterwarnings('ignore')

import time
import numpy as np
import pandas as pd

import streamz as sz

from IPython.display import display, Markdown
from bokeh.models.widgets import HTMLTemplateFormatter
import param
import holoviews as hv; hv.extension("bokeh", logo=False)
import panel as pn;     pn.extension()

<div style="float:center;width:100%;text-align: center;"><strong style="height:60px;color:darkred;font-size:40px;">Iterative Methods</strong></div>

# 1. Code: Monitor the Evolution of an Iterative Scheme

In [ ]:
def _table_formatter2D(plot, element):
    plot.handles['table'].columns[3].formatter = HTMLTemplateFormatter(
        template = """<div style="color:red;"><%= value ? value.toExponential(4) : value %></div>""")
def _table_formatterND(plot, element):
    plot.handles['table'].columns[2].formatter = HTMLTemplateFormatter(
        template = """<div style="color:red;"><%= value ? value.toExponential(4) : value %></div>""")

class GraphicalMonitor2D:
    """monitor the error evolution of an iterative scheme"""

    def __init__(self, sz=10, use_log=False):
        self.buffer  = hv.streams.Buffer(self._to_dataframe(), length=sz, index=False)
        self.use_log = use_log

        self.plots = pn.Column( pn.Row( hv.DynamicMap(self.display_xy,  streams=[self.buffer]),
                                        hv.DynamicMap(self.display_err, streams=[self.buffer])
                                      ),
                                 hv.DynamicMap(self.display_tbl, streams=[self.buffer])
                            )
    def _to_dataframe(self, data=None):
        data_def = {"step": int, "x": float, "y": float, "error": float}

        if data is None:
            return pd.DataFrame(
                {i: [] for i in data_def},
            ).astype(dtype=data_def)

        return pd.DataFrame([data], columns=data_def).astype(dtype=data_def)

    def reset_plots(self):
        self.buffer.clear()

    def monitor(self, data):
        self.buffer.send(self._to_dataframe(data))

    def display_xy(self, data):
        if data.empty:
            return ( hv.Scatter([], "x", "y") * hv.Curve([], "x", "y")  * hv.Scatter([], "x", "y")
                    ).opts( width=500,  xticks=4, yticks=4, tools=["hover"], show_grid=True, title="Solution Estimate" )
        h_last_point = hv.Scatter( (data["x"].iloc[-1], data["y"].iloc[-1]), "x", "y" ).opts(size=10, color="red", tools=["hover"])

        h_points = hv.Curve((data["x"], data["y"]), "x", "y") *\
                   hv.Scatter( (data["x"], data["y"]), "x", "y" )\
                     .opts(color="darkblue", padding=0.05, size=8, tools=["hover"], show_grid=True)

        return (h_points.opts(xticks=4, yticks=4) * h_last_point).opts(width=500, tools=["hover"])

    def display_err( self, data):
        edim = hv.Dimension('error', range=(1e-18, np.nan))
        if data.empty:
            return hv.Curve([], "step", edim).opts( tools=["hover"], logy=self.use_log, yticks=4, show_grid=True, title="Error" )
        return hv.Curve((data["step"], data["error"]), "step", edim).opts(padding=0.05, xticks=4, logy=self.use_log)

    def display_tbl( self, data ):
        if data.empty:
            return hv.Table(data).opts(height=450,width=500, hooks=[_table_formatter2D])
        return hv.Table(data).opts(hooks=[_table_formatter2D])

class GraphicalMonitorND:
    """monitor the error evolution of an iterative scheme"""

    def __init__(self, dimension, sz=10, use_log=False):
        self._data_def = {"step": int}
        for i in range(dimension):
            self._data_def['x'+ str(i+1)] = float
        self._data_def["error"] = float

        self.buffer  = hv.streams.Buffer(self._to_dataframe(), length=sz, index=False)
        self.use_log = use_log

        self.plots = pn.Row( hv.DynamicMap(self.display_err, streams=[self.buffer]),
                             hv.DynamicMap(self.display_tbl, streams=[self.buffer]) )

    def _to_dataframe(self, data=None):
        if data is None:
            return pd.DataFrame(
                    {i: [] for i in self._data_def},
                ).astype(dtype= self._data_def)

        flattend_data = [data[0]] + data[1].tolist() + [data[2]]
        return pd.DataFrame([flattend_data], columns=self._data_def).astype(dtype=self._data_def)

    def reset_plots(self):
        self.buffer.clear()

    def monitor(self, data):
        self.buffer.send(self._to_dataframe(data))

    def display_err( self, data):
        edim = hv.Dimension('error', range=(1e-18, np.nan))
        if data.empty:
            return hv.Curve([], "step", edim).opts(  logy=self.use_log, tools=["hover"], yticks=4, show_grid=True, title="Error" )
        return hv.Curve((data["step"], data["error"]), "step", edim).opts(padding=0.05, xticks=4, logy=self.use_log, tools=["hover"])

    def display_tbl( self, data ):
        if data.empty:
            return hv.Table(data).opts(height=450,width=500, hooks=[_table_formatterND])
        return hv.Table(data).opts(hooks=[_table_formatterND])

# 2. Iterative Solutions of $\mathbf{A x = b}$

## 2.1 Idea: Set up a Fixed Point

**Wanted:** an iterative scheme to solve $A x = b$ for a square matrix $A$.

We split $A = S - T$ such that $S$ is invertible:

$\qquad \begin{align}
\color{blue}{\mathbf{A x = b}}   & \Leftrightarrow ( S - T )\ x = b \\
          & \Leftrightarrow S x = T x + b \\
          & \Leftrightarrow x = S^{-1} T x + S^{-1} b \\
          & \Leftrightarrow \color{blue}{\mathbf{x = \tilde{A}x + \tilde{b}}}
\end{align},$<br><br>
$\qquad$ where we have set $\tilde{A} = S^{-1} T, \;\; \tilde{b} = S^{-1} b.$

Convert this last equation into an iteration:<br>
$\qquad$ Start with some vector $x_0$, and compute
$\qquad \color{blue}{\mathbf{x_{n} = \tilde{A} x_{n-1} + \tilde{b}, \quad n=1,2,\dots}}$

The difference between successive iterates is $\qquad\;\; \color{blue}{\mathbf{e_{n} = x_n - x_{n-1}}}$.

When $\color{blue}{\mathbf{\underset{\mathbf{n \to \infty}}{\operatorname{\lim}} e_{n} = 0}}$, the iterative converges to a **fixed point**, a solution of $A x = b$

#### **Example**

> Consider $\begin{pmatrix} x_n \\ y_n \end{pmatrix} =  \begin{pmatrix}  0.4 & -0.1 \\ 0.2 & 0.1 \end{pmatrix}
\begin{pmatrix} x_{n-1} \\ y_{n-1} \end{pmatrix} + \begin{pmatrix} 0.8 \\ 1.6 \end{pmatrix},
\qquad \text{with} \quad \begin{pmatrix}x_0 \\ y_0 \end{pmatrix} = \begin{pmatrix} 3 \\ 1 \end{pmatrix}$

> A fixed point satisfies<br>
$\quad \begin{pmatrix} x \\ y \end{pmatrix} =  \begin{pmatrix}  0.4 & -0.1 \\ 0.2 & 0.1 \end{pmatrix}
\begin{pmatrix} x \\ y \end{pmatrix} + \begin{pmatrix} 0.8 \\ 1.6 \end{pmatrix}
\quad \Leftrightarrow \quad
\begin{pmatrix} 0.6 & 0.1 \\ -0.2 & 0.9 \end{pmatrix} 
\begin{pmatrix} x \\ y \end{pmatrix} =
\begin{pmatrix} 0.8 \\ 1.6 \end{pmatrix}
$

In [ ]:
def iterate_scheme( A, b, x, n, plots, tol=1.e-10 ):
    plots.reset_plots()

    errs = []

    twoD = A.shape[1] == 2
    if twoD:
        print( " Iteration\t\t x \t          y \t\t Error")
    else:
        print( " Iteration\t\t x \t          y \t          z \t\t Error")

    for i in range(n):
        x_old = x
        x     = A @ x + b
        err   = np.linalg.norm( x-x_old )

        errs.append( err )
        if twoD:
            print( f"{i:10}\t {x[0]: .10f}\t   {x[1]: .10f}\t{err: .10f}" )
            plots.monitor( [i, x[0], x[1], err] )
        else:
            with np.printoptions(formatter={'float': '{: 0.10f}'.format}):
                print( f"{i:10}\t {x}\t{err: .10f}" )
            plots.monitor( [i, x, err] )

        if err < tol:
            return x, err, errs
        time.sleep(0.5)
    return None,err,errs

## 2.2 Convergence

Given a scheme $x_n = A x_{n-1} + b$ with some initial guess $x_0$,<br>
$\qquad$ when does it converge?

Let's look at the change at each iteration step: $e_n = x_{n} - x_{n-1}$.<br>
$\qquad$ Starting with $n = 1,$ we see<br>
$\qquad\quad\begin{align}
e_2 &= \left(A x_1 + b \right) - \left( A x_0 + b \right) &= A e_1 & \\
e_3 &= \left(A x_2 + b \right) - \left( A x_1 + b \right) &= A e_2 & = A^2 e_1 \\
\dots & & & \\
e_{n+1} &= \left(A x_{n} + b \right) - \left( A x_{n-1} + b \right) &= A e_n & = A^{n} e_1 \\
\end{align}$

When $A = S \Lambda S^{-1}$, we obtain<br><br>
$\qquad
\mathbf{\underset{\mathbf{n \to \infty}}{\operatorname{\lim}}} e_{n+1} =
S \ \begin{pmatrix}
      \mathbf{\underset{\mathbf{n \to \infty}}{\operatorname{\lim}}} \lambda^n_1 & 0 & \dots & 0 \\
  0 & \mathbf{\underset{\mathbf{n \to \infty}}{\operatorname{\lim}}} \lambda^n_2     & \dots & 0 \\
    &                                                                           &   & \dots &   \\
  0 & 0 & \dots & \mathbf{\underset{\mathbf{n \to \infty}}{\operatorname{\lim}}} \lambda^n_N \\
\end{pmatrix}\ S^{-1}
$

We see that this limit goes to zero iff $\lvert \lambda_i \rvert < 1$ for each $i=1,2, \dots N$.

<div style="float:left;padding-left:5pt;width:98%;background-color:#F2F5A9;color:black;">

**Definition:** The **spectral radius** of a matrix $A$ of size $N \times N$ is given by<br>
$\qquad \rho(A) = \underset{\mathbf{i=1,2,\dots N}}{\operatorname{\max}}\ \lvert \lambda_i \rvert$

**Theorem:** the iteraitve scheme $x_n = A x_{n-1} + b$ converges iff $\rho (A) < 1$.
</div>

## 2.3 Jacobi Iteration

Let $A = S - T, $ where $S$ is the diagonal matrix $S_{i i} = A_{i i}, i=1, \dots N$ (the diagonal entries of $A$)

Then $A x = b \Leftrightarrow S x = T x + b \Leftrightarrow x = S^{-1} T x + S^{-1} b$

In [ ]:
A = np.array([[6., .1], [-.2, .9]]); b = np.array([0.8,1.6])
S = np.diag( np.diag(A) )
T = S - A
"A =";A
"b =";b

"S =";S

In [ ]:
Sinv = np.linalg.inv(S)
SinvT = Sinv @ T
Sinvb = Sinv @ b

Markdown('**<font size="5">Jacobi Iteration, 2D Example**</font>')
plots = GraphicalMonitor2D(100,use_log=False)
plots.plots
x,err,errs = iterate_scheme( SinvT, Sinvb, np.array([3,-2]),  100, plots, tol=1e-8 )

In [ ]:
A     = np.array([[ 6., 2, -3],
                  [-1., 3,  1],
                  [ 1., 2,  4]])
b = np.array([1., 1, 1])

S     = np.diag( np.diag(A) )
T     = S - A
Sinv  = np.linalg.inv(S)    # Note:  In our iteration, solve  S x_n = T x_{n-1} + b   for x_n  instead!!!!

SinvT = Sinv @ T
Sinvb = Sinv @ b

In [ ]:
eigA  = np.linalg.eigvals(SinvT)
print("Spectral Radius: ")
print(".  eigenvalues: ", np.round( eigA, 3) )
print(".  max(abs(e)): ", np.round(max(np.abs(eigA)),3))

In [ ]:
plots = GraphicalMonitorND(3,10,use_log=False)
Markdown('**<font size="5">Jacobi Iteration, 3D Example**</font>')

plots.plots
x,err,errsJ = iterate_scheme( SinvT, Sinvb, np.array([3,-2,1]), 100, plots, tol=1e-8 )

## 2.4 Gauss Seidel Iteration (GS)

**Idea:** Consider an iterative scheme<br>
$\qquad\begin{align}
x_{n} &=\;\; 4 \ x_{n-1} &+ 4 \ &y_{n-1} &+ 3 \ &z_{n-1} &+ 2 \\
y_{n} &=\;\; 2 \ x_{n-1} &- 2 \ &y_{n-1} &- 3 \ &z_{n-1} &- 1 \\
z_{n} &=\;\; 3 \ x_{n-1} &- 3 \ &y_{n-1} &+ 3 \ &z_{n-1} &+ 1 \\
\end{align}$

why not use the updated versions of $x_{n}, y_{n}$ as soon as they become available?<br>
$\qquad\begin{align}
x_{n} &= \;\; 4\ x_{n-1}            &+ 4 \ &y_{n-1}            &+ 3 \ &z_{n-1} &+ 2 \\
y_{n} &= \;\; 2\ x_{\color{red}{n}} &- 2 \ &y_{n-1}            &- 3 \ &z_{n-1} &- 1 \\
z_{n} &= \;\; 3\ x_{\color{red}{n}} &- 3 \ &y_{\color{red}{n}} &+ 3 \ &z_{n-1} &+ 1 \\
\end{align}$

**This corresponds to choosing $\mathbf{S}$ to be the <font color="red">lower triangular part of $\mathbf{A}$.</font>**

In [ ]:
S     = np.tril(A)
T     = S - A
Sinv  = np.linalg.inv(S)    # Note:  In our iteration, solve  S x_n = T x_{n-1} + b   for x_n  instead!!!!

SinvT = Sinv @ T
Sinvb = Sinv @ b

eigA  = np.linalg.eigvals(SinvT)
print("Spectral Radius: ")
print(".  eigenvalues: ", np.round(eigA,3) )
print(".  max(abs(e)): ", np.round( max(np.abs(eigA)),3 ))

In [ ]:
plots = GraphicalMonitorND(3,10,use_log=False)
Markdown('**<font size="5">Gauss Seidel Iteration**</font>')

plots.plots
x,errGS,errsGS = iterate_scheme( SinvT, Sinvb, np.array([3,-2,1]), 100, plots, tol=1e-8 )

## 2.5 Successive Overrelaxation (SOR)

**Idea:** A step of the Gauss-Seidel scheme moves the solution estimate form $x_{n-1}$ to $x_n$.<br>
$\quad$ How about moving by a different amount, i.e., for $x_{n-1}$ to $x_{n-1} + \alpha \left(x_{n}-x_{n-1}\right)$?

With $S = L$, $T=A-L$ i.e., the matrix split used for Gauss-Seidel, we have<br><br>
$\quad\begin{align}
\tilde{x}_{n} &= S^{-1}T x_{n-1} + S^{-1} b \\
x_n &= x_{n-1} + \alpha \left( \tilde{x}_n - x_{n-1} \right)
\end{align}$

Setting $\omega = 1-\alpha$, the sheme can be shown to be equivalent to<br><br>
$\quad \left( D+\omega L \right) x_{n} = \left( \left( 1-\omega \right) D - \omega U \right) x_{n-1} + \omega b$

# 3. Iterative Methods for $\mathbf{A x = \lambda x}$

## 3.1 Power Method

In [ ]:
def power_method( A, x, n, plots, tol=1.e-10 ):
    plots.reset_plots()
    print( " Iteration\t\t x \t          y \t\t Change")

    for i in range(n):
        x_old = x
        x     = A @ x
        x     = x / np.linalg.norm( x )

        err   = np.linalg.norm( x-x_old )

        print( f"{i:10}\t {x[0]: .10f}\t   {x[1]: .10f}\t{err: .10f}" )
        plots.monitor( [i, x[0], x[1], err] )

        if err < tol:
            return x, x.T @ A @ x

        time.sleep(0.3)
    return None

In [ ]:
plots = GraphicalMonitor2D(15,use_log=False)
Markdown('**<font size="5">Power Method**</font>')
plots.plots
power_method( np.array([[3,1],[5,2]]),
              np.array([1,1]),
              100,
              plots
            )

##### **Power Method with Streamz**

In [ ]:
# ================================================================================================
#  Streamz Pipeline Functions
# ================================================================================================
def make_Step(A):
    def step(x):
        x,diff2 = power_method_step( step.A, x )
        e_val = estimate_eigenvalue( step.A, x)
        return { 'e_val': e_val, 'e_vec': x, 'diff2': diff2}
    step.A=A
    return step
# --------------------------------------------------
def step_Counter( state, cur):
    state += 1

    cur['step_n'] = state
    return state,cur
# --------------------------------------------------
def make_ConvergenceCheck(n_max, tol=1e-6):
    def checkConvergence(cur):
        cur['cnv_flag'] = check_convergence( cur['diff2'], cur['step_n'], checkConvergence.n_max, checkConvergence.tol2 )
        return cur

    checkConvergence.tol2  = tol*tol
    checkConvergence.n_max = n_max
    return checkConvergence
# --------------------------------------------------
def make_LoopBack(n, pipeline):
    def loop( cur ):
        if loop.n > cur['step_n'] and cur['cnv_flag'] == 0:
            loop.p.emit(cur['e_vec'])
        return cur
    loop.n = n
    loop.p = pipeline
    return loop
# --------------------------------------------------
def make_Delay(t):     # the sz rate limit function appears to cause problems in jupyter
    def delay(cur):
        time.sleep(t)
        return cur
    return delay

In [ ]:
def power_method_step(A, x):
    '''implements a single step of the power method

    Args:
        A  : the np.array matrix
        x  : the current np.array unit vector
    Returns
        diff2, eigval_estimate, eigvec_estimate, where diff2 is the l2 norm of the change in the eigvec_estimate
    '''
    new_x        = A @ x

    normalized_x = new_x / np.linalg.norm(new_x)
    diff         = normalized_x - x

    return normalized_x, diff @ diff

def power_method_convergence_check(diff, n_step, n_max, tol):
    '''power method convergence

    Args:
        n_step:  the current step number
        diff:    convergence estimate
        tol:     tolerance
        n_max:   maximum number of steps
    Returns:
        1 if converged, -1 if too many steps, 0 otherwise
    '''
    return 1 if diff < tol else -1 if n_step >= n_max else 0

def estimate_eigenvalue( A, x_hat ):
    '''eigenvalue estimate using the Rayleigh coefficient

    Args:
        A:       matrix
        dix_hat: unit length eigenvector estimate
    Returns:
        estimated eigenvalue
    '''

    return np.dot( x_hat, A @ x_hat )

def check_convergence(diff2, n_step, n_max, tol2):
    '''power method convergence

    Args:
        n_step:  the current step number
        cur:     current output of step(A, x)
        tol:     tolerance
        n_max:   maximum number of steps
    Returns:
        1 if converged, -1 if too many steps, 0 otherwise
    '''
    #print( 'diff2', diff2, ', cnv?', diff2<tol2, '; n_step', n_step,' < ', n_max)
    return 1 if diff2 < tol2 else -1 if n_step >= n_max else 0

In [ ]:
# ================================================================================================
#  Streamz Pipeline Sinks
# ================================================================================================
def make_SaveResults():
    def save(cur):
        save.results.append(cur)

    save.results = []
    return save
# --------------------------------------------------
def show_ppresults(x):
    with np.printoptions(formatter={'float': '{: 0.4f}'.format}):
        print( f'''{x["step_n"]:3}:   {x["e_vec"]},  diff2: {x["diff2"]:.8g}''' )

In [ ]:
# Holoviews Output Streams
# --------------------------------------------------
def eigenvalue_plot( buffer_size = 100 ):
    import math

    eigenvalue_estimates = pd.DataFrame({'iteration': [], 'eigenvalue': [], 'diff_squared': []}, columns=['iteration', 'eigenvalue', 'diff_squared'])
    eigenvalue_stream    = hv.streams.Buffer(eigenvalue_estimates, length=buffer_size, index=False)

    #diff2_dim = hv.Dimension( "diff_squared", range=(1e-18, np.nan))
    def plot(data):
        h = hv.Curve(data, 'iteration', 'eigenvalue',   label = 'Evolution of the Eigenvalue')+\
            hv.Curve(data, 'iteration', 'diff_squared', label = 'Change in the Eigenvector')
        return h

    h =\
    hv.DynamicMap( plot, streams=[eigenvalue_stream] )\
      .relabel('Evolution of the Eigenvalue')

    return eigenvalue_stream, h.opts({'Curve':{'width':500, 'show_grid':True, 'logy':True} }).relabel('')
# ----------------------------------------------------------
def make_StreamToHoloviews( eigenvalue_stream ):
    def stream_ToHoloviews(x):
        eigenvalue_stream.send( pd.DataFrame({'iteration':[x['step_n']], 'eigenvalue': [x['e_val']], 'diff_squared':[np.abs(x['diff2'])]},
                                         columns=['iteration', 'eigenvalue','diff_squared']) )
    return stream_ToHoloviews

In [ ]:
A = np.array([[12., 1., -1.],
              [20., 2.,  1.],
              [ 0., 0., 30.]])

pipeline = sz.Stream(loop=None)
pm_out   = pipeline.map(make_Step(A))\
              .accumulate( step_Counter, returns_state=True, start=0)\
              .map(make_ConvergenceCheck(50,tol=1e-6))
              #.rate_limit(0.5) #appears not to work in jupyter notebook :-(
pm_out.sink(make_LoopBack(50,pipeline));
pm_out.sink(show_ppresults);
save = make_SaveResults()
pm_out.sink(save);

eigenvalue_stream, evolution_plot = eigenvalue_plot()
pm_out.sink( make_StreamToHoloviews(eigenvalue_stream) );

display(pipeline.visualize(rankdir="LR"));

In [ ]:
evolution_plot

In [ ]:
pipeline.emit(np.array([1.,2.,1.]))

In [ ]:
save.results[0]

In [ ]:
df =pd.DataFrame(
    {'step_n': [i['step_n'] for i in save.results ],
     'e_val':  [i['e_val' ] for i in save.results ],
     'e_vec':  [i['e_vec' ] for i in save.results ],
     'diff2':  [i['diff2' ] for i in save.results ]})
df.head(3)

## 3.2 Inverse Power Method

In [ ]:
import scipy.linalg as sla

In [ ]:
def inverse_power_method( A, x, n, plots, tol=1.e-10 ):
    plots.reset_plots()
    print( " Iteration\t\t x \t          y \t\t Change")

    lu = sla.lu_factor(A)

    for i in range(n):
        x_old = x
        x     = sla.lu_solve( lu, x)        # new_x = A^.inv x <=> solve A new_x = x
        x     = x / np.linalg.norm( x )

        err   = np.linalg.norm( x-x_old )

        print( f"{i:10}\t {x[0]: .10f}\t   {x[1]: .10f}\t{err: .10f}" )
        plots.monitor( [i, x[0], x[1], err] )

        if err < tol:
            return x, x.T @ A @ x   # return an eigenpair, lambda estimated using Rayleigh Coefficient

        time.sleep(0.3)
    return None

In [ ]:
A = np.array([[12., 1., -1.],
              [20., 2.,  1.],
              [ 0., 0., 30.]])

plots = GraphicalMonitor2D(15,use_log=False)
Markdown('**<font size="5">Power Method**</font>')
plots.plots

inverse_power_method( A,
                np.array([1,1,1]),
                100,
                plots
              )

##### **Implementation using streamz**

In [ ]:
def inverse_power_method_step( lu, x):
    '''implements a single step of the power method

    Args:
        A  : the np.array matrix
        x  : the current np.array unit vector

    Returns
        diff2, eigval_estimate, eigvec_estimate, where diff2 is the l2 norm of the change in the eigvec_estimate
    '''
    new_x        = sla.lu_solve( lu, x)   # new_x = A^.inv x <=> solve A new_x = x

    normalized_x = new_x / np.linalg.norm(new_x)
    diff         = normalized_x - x

    return normalized_x, diff @ diff

def make_InverseStep(A):
    def inverse_step(x):
        x,diff2 = inverse_power_method_step( inverse_step.lu, x )
        e_val   = estimate_eigenvalue( inverse_step.A, x)
        return { 'e_val': e_val, 'e_vec': x, 'diff2': diff2}
    inverse_step.lu = sla.lu_factor(A)
    inverse_step.A  = A
    return inverse_step

In [ ]:
pipeline = sz.Stream(loop=None)
pm_out   = pipeline.map(make_InverseStep(A))\
              .accumulate( step_Counter, returns_state=True, start=0)\
              .map(make_ConvergenceCheck(50,tol=1e-8))
#              .rate_limit(0.5) appears not to work in jupyter notebook :-(
pm_out.sink(make_LoopBack(50,pipeline));
pm_out.sink(show_ppresults);
save = make_SaveResults()
pm_out.sink(save);

eigenvalue_stream, evolution_plot = eigenvalue_plot()
pm_out.sink( make_StreamToHoloviews(eigenvalue_stream) );

display(pipeline.visualize(rankdir="LR"));

In [ ]:
evolution_plot

In [ ]:
pipeline.emit(np.array([1.,2.,1.]))